In [4]:
# imports key libraries
import pandas as pd
import numpy as np

link = 'C:/Users/HUAWEI/Desktop/Projects/Saint-James-Data-Analysis/Data/current_sheet.xlsx'

check_out_df = pd.read_excel(link, 'Check-outs')

col = check_out_df['CHECK -IN']

index = []
for i in range(len(col)):
    try:
        col[i].year
        index.append(i)
    except:
        pass
    
check_out_df = check_out_df.iloc[index,:].reset_index(drop=True)

C:\Users\HUAWEI\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [8]:
check_out_df

,CHECK -IN,CHECK OUT,NIGHTS,GUESTS+,ROOM,BED,NAME,BOOKING PRICE,PAID ON,Booking Type,KD CASH,KD CARD,NOTES:,STATUS,BOOKING CHECKED,Unnamed: 15,Unnamed: 16
0,2023-06-18,2023-07-01,13.0,1.0,21,21C,Alexander Mollison,523.00,2023-06-18 00:00:00,BB,NaN,10.0,NaN,paid,booking ok,NaN,NaN
1,2023-06-30,2023-07-01,1.0,2.0,2,2EF,pawar suvarna,105.14,2023-06-26 00:00:00,booking vc,10.0,NaN,NaN,PAID,booking ok,NaN,NaN
2,2023-06-25,2023-07-01,6.0,1.0,5,5H,Carlin noel,196.00,2023-06-25 00:00:00,walk in,NaN,10.0,NaN,paid,NaN,NaN,NaN
3,2023-06-30,2023-07-01,1.0,1.0,2,2D,Cindy Fay,42.08,2023-06-30 00:00:00,HW,NaN,10.0,NaN,Paid,booking ok,NaN,NaN
4,2023-06-30,2023-07-01,1.0,1.0,24,24A,Guerrero/Tadeo Francisco,51.73,NaN,trip vc,NaN,NaN,NaN,paid,booking ok,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,2023-07-30,2023-08-05,6.0,1.0,5,5A,Lina Torres,125.00,2023-07-30 00:00:00,Walk in,NaN,10.0,extention change room lower bunk,paid,booking ok,NaN,NaN
997,2023-08-04,2023-08-05,1.0,1.0,33,33B,bobby kiln,45.00,NaN,bb,NaN,10.0,NaN,Paid,booking ok,NaN,NaN
998,2023-09-06,2023-09-09,3.0,1.0,3,3C,Navkaran singh,96.30,NaN,booking VCD,NaN,NaN,NaN,charge vc on day,booking ok,NaN,NaN
999,2023-08-03,2023-08-05,2.0,1.0,33,33A,Ned Quealy,57.38,2023-08-03 00:00:00,HW,NaN,10.0,NaN,Paid,booking ok,NaN,NaN


In [25]:
clean_check_out_df = pd.DataFrame()

check_out_df = check_out_df[pd.isna(check_out_df.iloc[:,0:3]).sum(axis=1)==0].reset_index(drop=True)

def get_col(df, col_name, dot_method=None):
    col = df[col_name]
    values = []
    for i in range(len(col)):
        try:
            if dot_method != None:
                values.append(eval('df[col_name][i]' + dot_method))
            else:
                values.append(df[col_name][i])
        except:
            pass
        
    return [i for i in values if pd.isna(i) == False]

#
clean_check_out_df['In-Year'] = get_col(check_out_df, 'CHECK -IN', '.year')
clean_check_out_df['In-Month'] = get_col(check_out_df, 'CHECK -IN', '.month')
clean_check_out_df['In-Day'] = get_col(check_out_df, 'CHECK -IN', '.day')

import datetime as dt
import statistics as stats

nights = get_col(check_out_df, 'NIGHTS')
check_in = get_col(check_out_df, 'CHECK -IN')

check_out = []
for i in range(len(nights)):
    check_out.append(check_in[i] + dt.timedelta(days = nights[i]))
    
check_out = pd.DataFrame({'check_out': check_out})

clean_check_out_df['Out-Year'] = get_col(check_out, 'check_out', '.year')
clean_check_out_df['Out-Month'] = get_col(check_out, 'check_out', '.month')
clean_check_out_df['Out-Day'] = get_col(check_out, 'check_out', '.day')

clean_check_out_df['Nights'] = get_col(check_out_df, 'NIGHTS')
clean_check_out_df['Guests'] = check_out_df['GUESTS+'].fillna(1)
clean_check_out_df['Room'] = check_out_df['ROOM'].fillna(21)
clean_check_out_df['Price'] = check_out_df['BOOKING PRICE'].fillna(stats.median(check_out_df['BOOKING PRICE']))
clean_check_out_df['Booking Type'] = check_out_df['Booking Type'].str.lower().fillna('Booking VC')
clean_check_out_df['KD Cash'] = check_out_df['KD CASH'].fillna(0)
clean_check_out_df['KD Card'] = check_out_df['KD CARD'].fillna(0)
clean_check_out_df['Payment Status'] = check_out_df['STATUS'].str.lower().fillna('none')
clean_check_out_df['Booking Checked'] = [1 if i is not True else 0 for i in check_out_df['BOOKING CHECKED'].isna()]

by_months = clean_check_out_df['In-Month'].value_counts().iloc[0:6]
by_months = by_months[::-1]

by_nights = clean_check_out_df['Nights'].value_counts().sort_index().iloc[0:10]
by_guests = clean_check_out_df['Guests'].value_counts().sort_index().iloc[0:8]

by_room = clean_check_out_df['Room'].astype(str).value_counts().sort_index()

private = ['A', 'B', 'C', '30', '31', '37']
eight_bed = ['3', '4', '21', '24']
female = ['2', '5', '23']
six_bed = ['36', '32']
four_bed = ['33', '34', '35']
triple = ['1', '6', '20', '25']

room_type = []
for  i in range(len(by_room)):
    if by_room.index[i] in private:
        room_type.append('Private')
    elif by_room.index[i] in eight_bed:
        room_type.append('8 bed')
    elif by_room.index[i] in female:
        room_type.append('Female')
    elif by_room.index[i] in six_bed:
        room_type.append('6 bed')
    elif by_room.index[i] in four_bed:
        room_type.append('4 bed')
    elif by_room.index[i] in triple:
        room_type.append('Triple')
    else:
        room_type.append('Multiple')
    

room_df = pd.DataFrame({'Room': by_room.index, 'Count': by_room, 'Type': room_type}).reset_index(drop=True)

by_room = room_df.Count.sort_values(ascending=False)
by_room.index = room_df.Room

by_room_type = room_df[['Count', 'Type']].groupby('Type').sum().iloc[:,0].sort_values(ascending=True)
by_room_type = by_room_type[::-1]

by_price = clean_check_out_df.groupby('In-Month').sum()['Price'].iloc[0:6:]
by_price = round(by_price / 1000, 1)

clean_check_out_df['Booking Type'] = [x.strip() for x in clean_check_out_df['Booking Type']]

clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'booking', 'Booking Type'] = 'booking vc'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'bookin vc', 'Booking Type'] = 'booking vc'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'bookingvcd', 'Booking Type'] = 'booking vcd'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'expediavcn', 'Booking Type'] = 'expedia vcn'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'expediavcn', 'Booking Type'] = 'expedia vcn'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'group booking', 'Booking Type'] = 'group'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'trip', 'Booking Type'] = 'trip vc'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'trip.com', 'Booking Type'] = 'trip vc'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'walk ', 'Booking Type'] = 'walk-in'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'walk', 'Booking Type'] = 'walk-in'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'wak-in', 'Booking Type'] = 'walk-in'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'walk in', 'Booking Type'] = 'walk-in'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'walkin', 'Booking Type'] = 'walk-in'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'weekly rate', 'Booking Type'] = 'weekly'
clean_check_out_df.loc[clean_check_out_df['Booking Type'] == 'extention', 'Booking Type'] = 'extension'

by_booking_type = clean_check_out_df['Booking Type'].str.lower().value_counts().sort_values(ascending=False)

booking_provider = []
for i in range(len(clean_check_out_df)):
    if 'ooking' in clean_check_out_df['Booking Type'][i]:
        booking_provider.append('booking')
    elif 'expedia' in clean_check_out_df['Booking Type'][i]:
        booking_provider.append('expedia')
    elif 'trip' in clean_check_out_df['Booking Type'][i]:
        booking_provider.append('trip')
    else:
        booking_provider.append(clean_check_out_df['Booking Type'][i])
        
clean_check_out_df['Booking provider'] = booking_provider

by_booking_provider = clean_check_out_df['Booking provider'].value_counts().iloc[0:6]
by_booking_provider.index = by_booking_provider.index.str.title()

def to_numeric_kd_columns(col):
    for i in range(len(col)):
        try:
            col[i] > 0
        except:
            col[i] = 0
            
to_numeric_kd_columns(clean_check_out_df['KD Cash'])
to_numeric_kd_columns(clean_check_out_df['KD Card'])

by_KD_cash = clean_check_out_df[['In-Month', 'KD Cash']].groupby('In-Month').sum().iloc[0:6].squeeze()
by_KD_card = clean_check_out_df[['In-Month', 'KD Card']].groupby('In-Month').sum().iloc[0:6].squeeze()
by_KD_cash_ratio = (by_KD_cash / (by_KD_cash + by_KD_card))*100

paid_mispellings = ['paind', 'paid  ', 'paid by paynow', 'paud ', 'piad', 'paid paynow',
                    'paid with paynow', 'oaid', 'paid  paynow', 'paid   ', 
                    ' paid', 'paidl', 'paynow', 'paif', 'paid by pay now', 'paid ']

for i in range(len(clean_check_out_df)):
    if clean_check_out_df['Payment Status'][i] in paid_mispellings:
        clean_check_out_df['Payment Status'][i] = 'paid'
        
paid = []
not_paid = []

for i in range(len(clean_check_out_df)):
    if clean_check_out_df['Payment Status'][i] == 'paid':
        paid.append(1)
        not_paid.append(0)
    else:
        paid.append(0)
        not_paid.append(1)
        
clean_check_out_df['paid'] = paid
clean_check_out_df['not paid'] = not_paid

paid_months = clean_check_out_df[['In-Month', 'paid']].groupby('In-Month').sum().iloc[0:6].squeeze()
not_paid_months = clean_check_out_df[['In-Month', 'not paid']].groupby('In-Month').sum().iloc[0:6].squeeze()

by_payment_mistakes = round((not_paid_months / (paid_months+not_paid_months))*100,2)

clean_check_out_df['Booking not Checked'] = [1 if i is True else 0 for i in check_out_df['BOOKING CHECKED'].isna()]

checked = clean_check_out_df[['In-Month', 'Booking Checked']].groupby('In-Month').sum().iloc[0:6].squeeze()
not_checked = clean_check_out_df[['In-Month', 'Booking not Checked']].groupby('In-Month').sum().iloc[0:6].squeeze()

by_booking_checked = round((checked / (checked+not_checked))*100,2)

import datetime

def calculate_day_of_week(year, month, day):
    try:
        # Create a datetime object for the given date
        date_obj = datetime.datetime(year, month, day)
        # Get the day of the week as an integer (Monday is 0 and Sunday is 6)
        day_of_week = date_obj.weekday()
        
        return day_of_week
    except ValueError as e:
        return str(e)
    
weekdays = []
for i in range(len(clean_check_out_df)):
    year = clean_check_out_df['In-Year'][i]
    month = clean_check_out_df['In-Month'][i]
    day = clean_check_out_df['In-Day'][i]
    
    weekdays.append(calculate_day_of_week(year, month, day))
    
clean_check_out_df['weekday'] = weekdays

by_weekday = clean_check_out_df['weekday'].value_counts().sort_index()

C:\Users\HUAWEI\AppData\Local\Temp\ipykernel_12488\83375442.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_check_out_df['Payment Status'][i] = 'paid'


In [27]:
clean_check_out_df['In-Month']

0       6
1       6
2       6
3       6
4       6
       ..
996     7
997     8
998     9
999     8
1000    8
Name: In-Month, Length: 1001, dtype: int64